In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 메인 코드
data_list = []

# 드라이버 옵션 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach', True)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 웹 브라우저 띄우기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.maximize_window()

# 페이지 이동
url = "https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.CarType.A._.Service.EncarDiagnosis.)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
driver.get(url)
time.sleep(2)  # 페이지 로딩 시간

# 페이지 내의 모든 차량 링크를 찾기
car_links = driver.find_elements(By.CLASS_NAME, 'ItemBigImage_link_item__OiB1H')

if not car_links:
    print("No car links found. Please check the class name or page structure.")
else:
    print(f"Found {len(car_links)} car links.")

# 반복 횟수를 원하는 값으로 설정
반복_횟수 = 5  # 예시로 5로 설정

# 각 차량 링크에 접근하여 정보 수집
for index, link in enumerate(car_links):
    if index >= 반복_횟수:
        break
    # 링크를 클릭하여 상세 페이지로 이동
    try:
        link.click()
        time.sleep(2)  # 클릭 후 페이지 로딩 대기

        # 새로운 탭으로 포커스를 전환
        driver.switch_to.window(driver.window_handles[-1])
        
        # 상세 정보 추출 (예: 차량 이름, 가격 등)
        try:
            car_name = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'DetailHeader_title__kd6Pr'))
            ).text
            
            car_price = driver.find_element(By.CLASS_NAME, 'DetailLeadBottom_price__hogdk').text

            # 보험과 성능 점검 내역 추출
            insurance_history = driver.find_element(By.CSS_SELECTOR, '.DetailSummaryInsurance_insurance_warp__1ygX7').text

            # 옵션 추출
            options = driver.find_elements(By.CLASS_NAME, 'DetailOption_on__CfAJf')
            options_list = [option.text for option in options]
            
            # '자세히' 버튼 클릭 (JavaScript 사용)
            try:
                detail_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.DetailSummary_btn_detail__msm-h'))
                )
                driver.execute_script("arguments[0].click();", detail_button)
                time.sleep(2)  # '자세히' 버튼 클릭 후 대기
            except Exception as e:
                print(f"Error clicking detail button for car {index + 1}: {e}")

            # 연식, 주행거리, 연료, 변속기, 차종, 색상 등 세부 정보 추출
            car_year = driver.find_element(By.XPATH, '//dt[text()="연식"]/following-sibling::dd').text
            car_mileage = driver.find_element(By.XPATH, '//dt[text()="주행거리"]/following-sibling::dd').text
            car_fuel = driver.find_element(By.XPATH, '//dt[text()="연료"]/following-sibling::dd').text
            car_transmission = driver.find_element(By.XPATH, '//dt[text()="변속기"]/following-sibling::dd').text
            car_type = driver.find_element(By.XPATH, '//dt[text()="차종"]/following-sibling::dd').text
            car_color = driver.find_element(By.XPATH, '//dt[text()="색상"]/following-sibling::dd').text
            
            # 성능 점검 내역 추출
            performance_check = driver.find_element(By.CLASS_NAME, 'DetailInspect_wrap__gVdGB').text

            # 엔카진단 여부 확인
            try:
                encar_diagnosis_element = driver.find_element(By.CLASS_NAME, 'ServiceTag-module_encar_diagnosis__AHTTa')
                encar_diagnosis = 'O'
            except:
                encar_diagnosis = 'X'
            
            # 추출한 정보를 딕셔너리 형태로 저장
            car_data = {
                'name': car_name,
                'price': car_price,
                'year': car_year,
                'mileage': car_mileage,
                'fuel': car_fuel,
                'color': car_color,
                'type': car_type,
                'transmission': car_transmission,
                'encar_diagnosis': encar_diagnosis,
                'insurance_history': insurance_history,
                'performance_check': performance_check,
                'options': options_list
            }
            
            # 리스트에 저장
            data_list.append(car_data)
            print(f"Extracted data for car {index + 1}")
        except Exception as e:
            print(f"Error extracting data for car {index + 1}: {e}")
        
        # 현재 탭 닫기
        driver.close()
        
        # 처음 탭으로 포커스를 전환
        driver.switch_to.window(driver.window_handles[0])
        
        # 페이지 이동 사이에 약간의 대기 시간 추가
        time.sleep(2)
    except Exception as e:
        print(f"Error processing link {index + 1}: {e}")

# 결과를 데이터프레임으로 변환
df = pd.DataFrame(data_list)
print(df)

# 드라이버 종료
driver.quit()

# 데이터프레임을 CSV 파일로 저장
df.to_csv('car_data.csv', index=False)


Found 210 car links.
Extracted data for car 1
Extracted data for car 2
Extracted data for car 3
Extracted data for car 4
Extracted data for car 5
      name    price year mileage fuel color type transmission encar_diagnosis  \
0  64루1732  1,300만원  N/A     N/A  N/A   N/A  N/A          N/A               O   
1  29보5921    930만원  N/A     N/A  N/A   N/A  N/A          N/A               O   
2  10누0472    970만원  N/A     N/A  N/A   N/A  N/A          N/A               O   
3  15저9659    509만원  N/A     N/A  N/A   N/A  N/A          N/A               O   
4  64오1401  5,390만원  N/A     N/A  N/A   N/A  N/A          N/A               O   

             insurance_history performance_check               options  
0  ·보험이력\n6건\n·성능점검내역\n일반/엔카직영                    [, , , , , , , , , ]  
1  ·보험이력\n5건\n·성능점검내역\n일반/엔카직영                        [, , , , , , , ]  
2  ·보험이력\n1건\n·성능점검내역\n일반/엔카직영                      [, , , , , , , , ]  
3  ·보험이력\n6건\n·성능점검내역\n일반/엔카직영                        [, , , , , , , ]  
4 

In [7]:
df.head()

,name,price,year,mileage,fuel,color,type,transmission,encar_diagnosis,insurance_history,performance_check,options
0,64루1732,"1,300만원",N/A,N/A,N/A,N/A,N/A,N/A,O,·보험이력\n6건\n·성능점검내역\n일반/엔카직영,,"[, , , , , , , , , ]"
1,29보5921,930만원,N/A,N/A,N/A,N/A,N/A,N/A,O,·보험이력\n5건\n·성능점검내역\n일반/엔카직영,,"[, , , , , , , ]"
2,10누0472,970만원,N/A,N/A,N/A,N/A,N/A,N/A,O,·보험이력\n1건\n·성능점검내역\n일반/엔카직영,,"[, , , , , , , , ]"
3,15저9659,509만원,N/A,N/A,N/A,N/A,N/A,N/A,O,·보험이력\n6건\n·성능점검내역\n일반/엔카직영,,"[, , , , , , , ]"
4,64오1401,"5,390만원",N/A,N/A,N/A,N/A,N/A,N/A,O,·보험이력\n2건\n·성능점검내역\n일반/엔카직영,,"[, , , , , , , , , ]"
